In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import mne


In [2]:
import numpy as np

In [3]:
%matplotlib notebook


In [4]:
eeg_files = sorted(glob.glob('./data/0114ssEEG/*.csv'))

In [5]:
eeg_files

['./data/0114ssEEG/SDS00001.csv',
 './data/0114ssEEG/SDS00002.csv',
 './data/0114ssEEG/SDS00003.csv',
 './data/0114ssEEG/SDS00004.csv',
 './data/0114ssEEG/SDS00005.csv',
 './data/0114ssEEG/SDS00006.csv',
 './data/0114ssEEG/SDS00007.csv',
 './data/0114ssEEG/SDS00008.csv',
 './data/0114ssEEG/SDS00009.csv',
 './data/0114ssEEG/SDS00010.csv',
 './data/0114ssEEG/SDS00011.csv',
 './data/0114ssEEG/SDS00012.csv',
 './data/0114ssEEG/SDS00013.csv']

In [6]:
all_eeg_df = [(os.path.basename(df_path).replace("SDS", "").replace(".csv", "").replace("000", ""), pd.read_csv(df_path, header=11)) for df_path in eeg_files]

In [7]:
for df in all_eeg_df:
    print(df[1].head())

   Second  Volt  Volt.1
0  0.0000  2.48    0.82
1  0.0002  2.48    0.80
2  0.0004  2.48    0.76
3  0.0006  2.48    0.76
4  0.0008  2.48    0.70
   Second  Volt  Volt.1
0   0.000  -3.2   -0.48
1   0.001   0.0   -1.36
2   0.002  -3.2   -0.96
3   0.003  -3.2    0.24
4   0.004   0.0    1.68
   Second   Volt  Volt.1
0   0.000 -0.288    1.60
1   0.001 -0.312   -1.44
2   0.002 -0.312   -1.40
3   0.003 -0.336   -0.28
4   0.004 -0.368    0.56
   Second   Volt  Volt.1
0   0.000 -0.208    0.28
1   0.001 -0.256   -0.68
2   0.002 -0.272   -1.68
3   0.003 -0.304    1.24
4   0.004 -0.336    1.48
   Second   Volt  Volt.1
0   0.000 -0.328   -1.12
1   0.001 -0.352    0.36
2   0.002 -0.320    0.88
3   0.003 -0.248    0.72
4   0.004 -0.232   -0.08
   Second   Volt  Volt.1
0   0.000 -0.360    0.76
1   0.001 -0.320   -1.04
2   0.002 -0.288   -1.28
3   0.003 -0.248   -0.76
4   0.004 -0.256    0.08
   Second   Volt  Volt.1
0   0.000 -0.288    1.04
1   0.001 -0.312   -1.32
2   0.002 -0.304   -1.28
3   0.003 -0

In [8]:
recording_length = 70
sampling_rate = int(len(all_eeg_df[0][1]) / recording_length )

In [9]:
sampling_rate

1000

- Reading data into MNE

In [10]:
eeg_data = all_eeg_df[0][1].drop(columns=['Second'], errors='ignore').to_numpy().T  # Transpose to shape (n_channels, n_samples)


In [11]:
eeg_data

array([[ 2.48,  2.48,  2.48, ..., -2.5 , -2.5 , -2.5 ],
       [ 0.82,  0.8 ,  0.76, ..., -0.4 , -0.36, -0.38]], shape=(2, 70000))

In [12]:
    eeg_data = eeg_data / np.sqrt(np.nanmedian(eeg_data**2, axis=1, keepdims=True)).astype(np.float32)


In [13]:
eeg_data

array([[ 0.99999999,  0.99999999,  0.99999999, ..., -1.00806451,
        -1.00806451, -1.00806451],
       [ 1.41379314,  1.37931038,  1.31034487, ..., -0.68965519,
        -0.62068967, -0.65517243]], shape=(2, 70000))

In [14]:
fmin = 1
fmax = 30
start = 20
duration = 5
scaling = 50e-3

In [15]:
eeg_data

array([[ 0.99999999,  0.99999999,  0.99999999, ..., -1.00806451,
        -1.00806451, -1.00806451],
       [ 1.41379314,  1.37931038,  1.31034487, ..., -0.68965519,
        -0.62068967, -0.65517243]], shape=(2, 70000))

In [16]:
sampling_rate * 5

5000

In [17]:
eeg_data.shape

(2, 70000)

In [23]:
# Define the phases
wait_duration = 15  # seconds
stir_duration = 5   # seconds
rest_duration = 5   # seconds
off_set_time = 5    # seconds


In [27]:
index_to_mne = {}
for index, df in all_eeg_df:
    # Converting dataframe to Numpy array
    eeg_data = df.drop(columns=['Second'], errors='ignore').to_numpy().T  # Transpose to shape (n_channels, n_samples)   

    # Removing first 5 seconds and last 5 seconds of data
    # eeg_data = eeg_data[:, sampling_rate * 10: -sampling_rate * 10]

    # channel_names = df.columns[1:]
    channel_names = ["C3", "C4"]

    info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types='eeg')
    raw = mne.io.RawArray(eeg_data, info)
    
    # raw.apply_function(lambda x: x * 1e-5)

    # Load a standard montage
    montage = mne.channels.make_standard_montage('standard_1020')

    # Apply the montage to the raw object
    raw.set_montage(montage)

    # fig = raw.plot(scalings='auto', title=f"Raw Signal - {index}", start=start, duration=duration);
    # fig = raw.plot(scalings=dict(eeg=scaling), title=f"Raw Signal - {index}", start=start, duration=duration);

    # fig.suptitle(f"Raw Signal - {index}", fontsize=16)  # Add a title
    # fig.savefig(f"./plots/raw_signal_{index}.png")

    fig = raw.compute_psd(fmin=2, fmax=60, ).plot(average=False, spatial_colors=True, );
    fig.savefig(f"./plots/raw_power_{index}.png")


    ## Preprocessing

    # set notch filter to remove electrical noise
    raw.notch_filter(60)
    # only keeping frequencies between 1-50 Hz
    raw.filter(l_freq=fmin, h_freq=fmax, method="fir", fir_design="firwin", pad="reflect_limited") 
    
    # Downsample a lot
    raw.resample(128, npad='auto')

    raw = raw.crop(tmin=5, tmax=65)

    # Extract data for specific channels (e.g., the first 2 channels)
    data = raw.get_data()  # Shape: (len(channels), n_times)
    times = raw.times  # Time points in seconds

    # Normalize the data
    normalized_data = data / np.sqrt(np.nanmedian(data**2, axis=1, keepdims=True)).astype(np.float32)
    raw._data = normalized_data  # Overwrite data in MNE object

    # Plot each channel
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(times + 5, raw._data[0], 'orange')
    ax2.plot(times + 5, raw._data[1], 'blue')

    ax1.set_xlabel('X data')
    ax1.set_ylabel('Y1 data', color='g')
    ax2.set_ylabel('Y2 data', color='b')


    current_time = wait_duration
    action_segments = []

    while current_time < times[-1]:
        # action_segments.append((current_time, current_time + stir_duration))
        plt.axvspan(current_time, current_time + stir_duration, color='orange', alpha=0.3)
        current_time += stir_duration + rest_duration


    # plt.title("EEG Signal")
    # plt.xlabel("Time (s)")
    # plt.ylabel("Amplitude (µV)")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"./plots/filtered_signal_{index}.png")

    # fig = raw.plot(scalings='auto', title=f"Raw Signal - {index}", start=start, duration=duration);
    # fig = raw.plot(scalings=dict(eeg=scaling), title=f"Raw Signal - {index}", start=start, duration=duration);
    # fig.suptitle(f"Filtered Signal - {index}", fontsize=16)  # Add a title
    # fig.savefig(f"./plots/filtered_signal_{index}.png")

    fig = raw.compute_psd(fmin=fmin, fmax=fmax, ).plot(average=False, spatial_colors=True, );
    fig.savefig(f"./plots/filtered_power_{index}.png")

    # set up and fit the ICA
    ica = mne.preprocessing.ICA(n_components=2, method='fastica')
    ica.fit(raw)
    # ica.exclude = [1, 2]  # details on how we picked these are omitted here
    fig = ica.plot_properties(raw)[0]
    fig.savefig(f"./plots/ica0_{index}.png")

    fig = ica.plot_properties(raw)[1]
    fig.savefig(f"./plots/ica1_{index}.png")

    index_to_mne[index] = raw



Creating RawArray with float64 data, n_channels=2, n_times=70000
    Range : 0 ... 69999 =      0.000 ...    69.999 secs
Ready.
Effective window size : 2.048 (s)
Plotting power spectral density (dB=True).


<IPython.core.display.Javascript object>

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 6601 samples (6.601 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge

<IPython.core.display.Javascript object>

Effective window size : 16.000 (s)
Plotting power spectral density (dB=True).


...

Fitting ICA to data using 2 channels (please be patient, this may take a while)
Selecting by number: 2 components
Fitting ICA took 0.0s.


<IPython.core.display.Javascript object>

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


<IPython.core.display.Javascript object>

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [19]:
import matplotlib.pyplot as plt

for index, df in all_eeg_df:
    # Set 'Time' column as the x-axis
    time = df['Second']

    # Create a figure and axis
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot the first channel on the left y-axis
    channel_1 = df.columns[1]
    ax1.plot(time, df[channel_1], label=channel_1, color='b', alpha=0.7)
    ax1.set_ylabel(f'{channel_1} Amplitude (µV)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.set_xlabel('Time (s)')
    ax1.grid(True)

    # Create a second y-axis for the second channel
    ax2 = ax1.twinx()
    channel_2 = df.columns[2]
    ax2.plot(time, df[channel_2], label=channel_2, color='r', alpha=0.7)
    ax2.set_ylabel(f'{channel_2} Amplitude (µV)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # Add a title and legend
    plt.title(f'EEG Data {index}')
    
    plt.xlim(10, 20)
    
    fig.tight_layout()  # Adjust layout to prevent overlap



    plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# for index, df in all_eeg_df:
#     # Set 'Time' column as the x-axis
#     time = df['Second']

#     # Plot each EEG channel
#     plt.figure(figsize=(12, 6))
#     for column in df.columns[1:]:  # Skip 'Time' column
#         plt.plot(time, df[column], label=column, alpha=0.5)

#     # Customize the plot
#     plt.title('EEG Data {}'.format(index))
#     plt.xlabel('Time (s)')
#     plt.ylabel('Amplitude (µV)')
#     plt.legend()
#     plt.grid(True)

#     # Show the plot
#     plt.show()

In [21]:
# # Set 'Time' column as the x-axis
# time = eeg_df['Second']

# # Plot each EEG channel
# plt.figure(figsize=(12, 6))
# for column in eeg_df.columns[1:]:  # Skip 'Time' column
#     plt.plot(time, eeg_df[column], label=column, alpha=0.5)

# # Customize the plot
# plt.title('EEG Data')
# plt.xlabel('Time (s)')
# plt.ylabel('Amplitude (µV)')
# plt.legend()
# plt.grid(True)

# # Show the plot
# plt.show()